In [ ]:
!pip install elasticsearch6
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 5.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
from elasticsearch6 import Elasticsearch

timestamp = []
count = []
# Create an Elasticsearch client
client = Elasticsearch("http://10.99.163.194:30626/")

# Specify the index name
index_name = "*"

# Define the search query
query = {
    "size": 0,
    "query": {
        "bool": {
            "must": [
                {"exists": {"field": "message"}},
                {"exists": {"field": "@timestamp"}},
                {"exists": {"field": "kubernetes.pod_name"}},
                {"exists": {"field": "kubernetes.host"}},
                {"range": {"@timestamp": {"gte": "2023-06-19T17:30:00", "lt": "2023-06-19T19:30:00"}}}
            ]
        }
    },
    "aggs": {
        "logs_per_minute": {
            "date_histogram": {
                "field": "@timestamp",
                "interval": "minute"
            }
        }
    }
}

# Execute the search query
response = client.search(index=index_name, body=query)

# Process the aggregation results
aggregations = response["aggregations"]["logs_per_minute"]["buckets"]
for aggregation in aggregations:
    timestamp.append(aggregation["key_as_string"])
    count.append(aggregation["doc_count"])
    #print(f"Timestamp: {timestamp}, Log Count: {count}")


In [ ]:
el_agg_test = pd.DataFrame()
el_agg_test['timestamp'] = timestamp
el_agg_test['log_count'] = count
el_agg_test

,timestamp,log_count
0,2023-06-19T17:30:00.000Z,9769
1,2023-06-19T17:31:00.000Z,13375
2,2023-06-19T17:32:00.000Z,11841
3,2023-06-19T17:33:00.000Z,12263
4,2023-06-19T17:34:00.000Z,12307
...,...,...
115,2023-06-19T19:25:00.000Z,1665
116,2023-06-19T19:26:00.000Z,1678
117,2023-06-19T19:27:00.000Z,1896
118,2023-06-19T19:28:00.000Z,1923


In [ ]:
el_agg_test.to_csv('el_agg_test.csv')

In [ ]:
from elasticsearch6 import Elasticsearch

timestamp = []
count = []
# Create an Elasticsearch client
client = Elasticsearch("http://10.99.163.194:30626/")

# Specify the index name
index_name = "*"

# Define the search query
query = {
    "size": 0,
    "query": {
        "bool": {
            "must": [
                {"exists": {"field": "message"}},
                {"exists": {"field": "@timestamp"}},
                {"exists": {"field": "kubernetes.pod_name"}},
                {"exists": {"field": "kubernetes.host"}},
                {"range": {"@timestamp": {"gte": "2023-06-29T09:20:00", "lt": "2023-06-29T09:30:00"}}}
            ]
        }
    },
    "aggs": {
        "logs_per_minute": {
            "date_histogram": {
                "field": "@timestamp",
                "interval": "minute"
            }
        }
    }
}

# Execute the search query
response = client.search(index=index_name, body=query)

# Process the aggregation results
aggregations = response["aggregations"]["logs_per_minute"]["buckets"]
for aggregation in aggregations:
    timestamp.append(aggregation["key_as_string"])
    count.append(aggregation["doc_count"])
    #print(f"Timestamp: {timestamp}, Log Count: {count}")

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/elasticsearch6/connection/http_urllib3.py", line 232, in perform_request
    response = self.pool.urlopen(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 798, in urlopen
    retries = retries.increment(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/retry.py", line 525, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "/usr/loc

ConnectionError: ignored

In [ ]:
el_agg_train = pd.DataFrame()
el_agg_train['timestamp'] = timestamp
el_agg_train['log_count'] = count
el_agg_train

,timestamp,log_count
0,2023-06-19T19:30:00.000Z,1842
1,2023-06-19T19:31:00.000Z,1467
2,2023-06-19T19:32:00.000Z,1756
3,2023-06-19T19:33:00.000Z,1521
4,2023-06-19T19:34:00.000Z,1840
...,...,...
2755,2023-06-21T17:25:00.000Z,1733
2756,2023-06-21T17:26:00.000Z,1609
2757,2023-06-21T17:27:00.000Z,1834
2758,2023-06-21T17:28:00.000Z,1552


In [ ]:
el_agg_train.to_csv('el_agg_train0.csv')

In [ ]:
## Individual logs queries

In [ ]:
from elasticsearch6 import Elasticsearch

# Create an Elasticsearch client
client = Elasticsearch("http://10.99.163.194:30626/")

# Specify the index name
index_name = "*"
message = []
timestamp = []
pod_name = []
host = []
# Define the search query
query = {
    "size": 10000,
    "_source": ["message", "@timestamp", "kubernetes.pod_name", "kubernetes.host"],
    "query": {
        "bool": {
            "must": [
                {"exists": {"field": "message"}},
                {"exists": {"field": "@timestamp"}},
                {"exists": {"field": "kubernetes.pod_name"}},
                {"exists": {"field": "kubernetes.host"}},
                {"range": {"@timestamp": {"gte": "2023-06-19T19:30:00", "lt": "2023-06-21T17:30:00"}}}
            ]
        }
    },
    "sort": [{"@timestamp": "asc"}]
}

# Execute the initial search with scroll API
response = client.search(index=index_name, body=query, scroll="5m")
scroll_id = response["_scroll_id"]

while True:
    # Process the current page of results
    hits = response["hits"]["hits"]
    for hit in hits:
        source = hit["_source"]
        message.append(source["message"])
        timestamp.append(source["@timestamp"])
        pod_name.append(source["kubernetes"]["pod_name"])
        host.append(source["kubernetes"]["host"])
        #print(f"Message: {message}, Timestamp: {timestamp}, Pod Name: {pod_name}, Host: {host}")

    # Check if there are more pages to scroll
    if len(hits) > 0:
        response = client.scroll(scroll_id=scroll_id, scroll="5m")
        scroll_id = response["_scroll_id"]
    else:
        break

# Clear the scroll context
client.clear_scroll(scroll_id=scroll_id)

{'succeeded': True, 'num_freed': 19}

In [ ]:
el_train = pd.DataFrame()
el_train['timestamp'] = timestamp
el_train['message'] = message
el_train['pod_name'] = pod_name
el_train['host'] = host
el_train

,timestamp,message,pod_name,host
0,2023-06-19T19:30:00.054150460+00:00,"10.244.0.0 - - [20/Jun/2023:03:29:59 +0800] ""P...",ts-ui-dashboard-579c4974d4-b5tw2,worker-node-3
1,2023-06-19T19:30:00.059897640+00:00,E0619 19:29:59.477507 1 reflector.go:156...,kube-state-metrics-584cf6b78f-5pz4f,worker-node-3
2,2023-06-19T19:30:00.061128320+00:00,E0619 19:29:59.478287 1 reflector.go:156...,kube-state-metrics-584cf6b78f-5pz4f,worker-node-3
3,2023-06-19T19:30:00.062033436+00:00,E0619 19:29:59.479064 1 reflector.go:156...,kube-state-metrics-584cf6b78f-5pz4f,worker-node-3
4,2023-06-19T19:30:00.062816375+00:00,E0619 19:29:59.480087 1 reflector.go:156...,kube-state-metrics-584cf6b78f-5pz4f,worker-node-3
...,...,...,...,...
4803927,2023-06-21T17:29:59.853299522+00:00,"10.244.0.0 - - [22/Jun/2023:01:29:58 +0800] ""P...",ts-ui-dashboard-579c4974d4-b5tw2,worker-node-3
4803928,2023-06-21T17:29:59.853594758+00:00,"10.244.0.0 - - [22/Jun/2023:01:29:59 +0800] ""P...",ts-ui-dashboard-579c4974d4-b5tw2,worker-node-3
4803929,2023-06-21T17:29:59.854060918+00:00,"10.244.0.0 - - [22/Jun/2023:01:29:59 +0800] ""P...",ts-ui-dashboard-579c4974d4-b5tw2,worker-node-3
4803930,2023-06-21T17:29:59.854368338+00:00,"10.244.0.0 - - [22/Jun/2023:01:29:59 +0800] ""P...",ts-ui-dashboard-579c4974d4-b5tw2,worker-node-3


In [ ]:
el_train.to_csv('elastic_norm_train0.csv')

In [ ]:
from elasticsearch6 import Elasticsearch

# Create an Elasticsearch client
client = Elasticsearch("http://10.99.163.194:30626/")

# Specify the index name
index_name = "*"
message = []
timestamp = []
pod_name = []
host = []
# Define the search query
query = {
    "size": 10000,
    "_source": ["message", "@timestamp", "kubernetes.pod_name", "kubernetes.host"],
    "query": {
        "bool": {
            "must": [
                {"exists": {"field": "message"}},
                {"exists": {"field": "@timestamp"}},
                {"exists": {"field": "kubernetes.pod_name"}},
                {"exists": {"field": "kubernetes.host"}},
                {"range": {"@timestamp": {"gte": "2023-06-19T17:30:00", "lt": "2023-06-19T19:30:00"}}}
            ]
        }
    },
    "sort": [{"@timestamp": "asc"}]
}

# Execute the initial search with scroll API
response = client.search(index=index_name, body=query, scroll="5m")
scroll_id = response["_scroll_id"]

while True:
    # Process the current page of results
    hits = response["hits"]["hits"]
    for hit in hits:
        source = hit["_source"]
        message.append(source["message"])
        timestamp.append(source["@timestamp"])
        pod_name.append(source["kubernetes"]["pod_name"])
        host.append(source["kubernetes"]["host"])
        #print(f"Message: {message}, Timestamp: {timestamp}, Pod Name: {pod_name}, Host: {host}")

    # Check if there are more pages to scroll
    if len(hits) > 0:
        response = client.scroll(scroll_id=scroll_id, scroll="5m")
        scroll_id = response["_scroll_id"]
    else:
        break

# Clear the scroll context
client.clear_scroll(scroll_id=scroll_id)


{'succeeded': True, 'num_freed': 19}

In [ ]:
len(timestamp)

382998

In [ ]:
timestamp[-1]

'2023-06-19T19:29:59.112412127+00:00'

In [ ]:
el_test = pd.DataFrame()
el_test['timestamp'] = timestamp
el_test['message'] = message
el_test['pod_name'] = pod_name
el_test['host'] = host
el_test.head(20)

,timestamp,message,pod_name,host
0,2023-06-19T17:30:00.741979792+00:00,2023/06/19 17:30:00.073325 api.go:285: \t ...,tsdb-mysql-2,worker-node-1
1,2023-06-19T17:30:00.742485302+00:00,2023/06/19 17:30:00.073543 api.go:295: \t ...,tsdb-mysql-2,worker-node-1
2,2023-06-19T17:30:00.742742811+00:00,"11a5482d-39ff-4693-966b-8f495bca1bc8:1-194038,",tsdb-mysql-2,worker-node-1
3,2023-06-19T17:30:00.743042584+00:00,"1cd36358-ec68-4a56-b4a4-39651f9ae705:1-307,",tsdb-mysql-2,worker-node-1
4,2023-06-19T17:30:00.743267072+00:00,"38e7236d-735a-4485-a285-6acc99115835:1-25,",tsdb-mysql-2,worker-node-1
5,2023-06-19T17:30:00.743568716+00:00,"686340ab-5a63-4eb5-85ff-ea86ae994772:1-290813,",tsdb-mysql-2,worker-node-1
6,2023-06-19T17:30:00.743804843+00:00,"72f2c7ee-5070-4da7-8354-675f96f69206:1-29,",tsdb-mysql-2,worker-node-1
7,2023-06-19T17:30:00.744053514+00:00,"b43addba-3574-4a4d-8d6f-d01ebcc59e94:1-896,",tsdb-mysql-2,worker-node-1
8,2023-06-19T17:30:00.744378356+00:00,"c4b20fb1-653a-4482-b631-b10016f2dd22:1-120,",tsdb-mysql-2,worker-node-1
9,2023-06-19T17:30:00.744708742+00:00,"e403df2e-914a-46b2-864d-6297a33e7fb9:1-35,",tsdb-mysql-2,worker-node-1


In [ ]:
el_test.to_csv('el_test.csv')